In [1]:
file_name_beam = 'ID60_ALD Beam test'
file_name_resonator = 'ID61_ALD Resonator test'

In [2]:
import gdsfactory as gf
from itertools import chain, repeat
from blocks import *
from dowhen import do, when
from blocks import resonator_with_beam, doubly_clamped_beam, ring_resonator, l_corner
from functools import partial
width_list = [0.05, 0.1, 0.2, 0.3,0.4,0.5, 1, 2, 5]
length_list = [1000,500,300,100,50,20,10]

width_list_repeat = list(chain.from_iterable(repeat(width_list, 1)))

beam_spacing = 35
@gf.cell
def frame(box:gf.kdb.DBox, layers=((2, 6))):
    left, bottom, right, top = box.left, box.bottom, box.right, box.top
    size = min(right - left, top - bottom)
    corner_coords = [
        (left, bottom),
        (right, bottom),
        (left, top),
        (right, top),
    ]
    c = gf.Component()
    for layer in layers:
        corner_rt = c << l_corner(layer)
        corner_rb = c << l_corner(layer)
        corner_lt = c << l_corner(layer)
        corner_lb = c << l_corner(layer)

        corner_lb.dmove(corner_coords[0])
        corner_rb.dmirror_x()
        corner_rb.dmove(corner_coords[1])

        corner_lt.dmirror_y()
        corner_lt.dmove(corner_coords[2])

        corner_rt.dmirror_x()
        corner_rt.dmirror_y()
        corner_rt.dmove(corner_coords[3])

    return c
@gf.cell
def beam_with_anchor(width, length, support_length, anchor_size:tuple):
    c = gf.Component()
    beam = c << doubly_clamped_beam_with_round_support(width, length, support_length)
    anchor_1 = c << gf.components.rectangle(size=anchor_size, port_type='placement')
    anchor_2 = c << gf.components.rectangle(size=anchor_size, port_type='placement')
    anchor_1.connect("e3", beam.ports["w1"], allow_width_mismatch=True)
    anchor_2.connect("e1", beam.ports["e1"], allow_width_mismatch=True)
    label = text_outline('abcd',size=5, mask_layer='DEEP_ETCH')
    c.add_ref(label)
    c.flatten()
    return c
@gf.cell
def comb_drive_ald_test(gap):
    from math import ceil
    c = gf.Component()
    fingers = combdrive_fingers(fingers=19, finger_length=4, finger_gap=gap, thickness=0.5, base_thickness=0.1,a_c=1)
    total_length = 19 * 0.5 + (19 - 1) * gap
    fingers_ref = c << fingers
    truss_ref = c << truss(width=0.15, size=1,mxn=(ceil(total_length),3))
    truss_ref.connect('E1', fingers_ref.ports['w1'],allow_width_mismatch=True)

    bbox = c.bbox()
    c.add_polygon(gf.kdb.DPolygon(bbox).sized(0.3),layer='DEEP_ETCH')
    text_ref = c << text_outline(f"G{gap:.2f}",size=12,outline_width=0.5,with_mask=False,layer='WG')
    text_ref.movex(10)
    return c
@gf.cell
def resonator_ald_test(gap):
    xs = cross_section_with_mask(0.43, 1,radius=5)
    beam_spec = partial(doubly_clamped_beam, width=0.3, length=20)
    resonator_spec = partial(ring_resonator, radius=5,length_x=3,length_y=0,cross_section=xs,bend='bend_circular',gap=gap,x_span=20)
    c = gf.Component()
    resonator = c << resonator_spec()
    beam = beam_spec().copy()
    beam.remap_layers({'WG': (2, 0)})
    spacer = c << vertical_spacer(gap)
    beam = c << beam
    spacer.connect("p1", resonator.ports["p1"])
    beam.connect("s1", spacer.ports["p2"])
    c2 = gf.Component()
    c2 << gf.boolean(c,c,'not','DEEP_ETCH','DEEP_ETCH',(2,0))
    c2 << c.extract(layers=['WG'])
    c2.ports = beam.ports
    rec = gf.components.rectangle(size=(1,2),layer='WG',port_type='placement')
    rec1 = c2<< rec
    rec1.connect('e3',beam.ports['w1'],allow_width_mismatch=True,allow_layer_mismatch=True)
    
    rec2 = c2<< rec
    rec2.connect('e1',beam.ports['e1'],allow_width_mismatch=True,allow_layer_mismatch=True)
    text_ref = c2 << text_outline(f"G{gap:.2f}",size=12,outline_width=0.5,with_mask=False,layer='WG')
    text_ref.movex(20)
    return c2
@gf.cell
def resonator_ald_test2(gap):
    sec1 = gf.Section(
        width=0.43,
        offset=0,
        layer='WG',
        port_names=["o1", "o2"],
        name="core",
    )
    xs = gf.CrossSection(sections=[sec1], radius=5)
    # xs = cross_section_with_mask(0.43, 1,radius=5)
    beam_spec = partial(doubly_clamped_beam_with_round_support, width=0.3, length=20,support_length=0.2,create_mask=True)
    resonator_spec = partial(ring_resonator, radius=5,length_x=3,length_y=0,cross_section=xs,bend='bend_circular',gap=gap,x_span=20)
    c = gf.Component()
    resonator = c << resonator_spec()
    beam = beam_spec().copy()
    beam.remap_layers({'WG': (2, 0)},recursive=True)
    spacer = c << vertical_spacer(gap)
    beam = c << beam
    spacer.connect("p1", resonator.ports["p1"])
    beam.connect("s1", spacer.ports["p2"])
    c2 = gf.Component()
    c2 << gf.boolean(c,c,'not','DEEP_ETCH','DEEP_ETCH',(2,0))
    c2 << c.extract(layers=['WG'])
    c2.ports = beam.ports
    rec = gf.components.rectangle(size=(1,2),layer='WG',port_type='placement')
    rec1 = c2<< rec
    rec1.connect('e3',beam.ports['w1'],allow_width_mismatch=True,allow_layer_mismatch=True)
    
    rec2 = c2<< rec
    rec2.connect('e1',beam.ports['e1'],allow_width_mismatch=True,allow_layer_mismatch=True)
    text_ref = c2 << text_outline(f"G{gap:.2f}",size=12,outline_width=0.5,with_mask=False,layer='WG')
    text_ref.movex(20)
    return c2
# @gf.cell
def beam_array():
    stair_offset = 10
    y=0
    c = gf.Component()
    for length in length_list:
        for width in width_list_repeat:
            ref = c.add_ref(beam_with_anchor(width, length, 1, (20, 35)))
            ref.movey(y)
            y += beam_spacing
        y += stair_offset
    h_stair = [beam_spacing*len(width_list_repeat)+stair_offset]*len(length_list)
    from collections import deque
    length_list_deque = deque(length_list)
    start = length_list_deque.pop()
    width_list = deque([start])
    while length_list_deque:
        poped = length_list_deque.pop()
        total_width_used = sum(width_list)
        width_list.appendleft(poped-total_width_used)
    w_stair = list(width_list)
    # w_stair = [500, 200, 200, 100]
    stair_trench = stair(w_stair, h_stair, layer=(2,0)).copy()
    stair_trench.movex(-1000)
    stair_trench.movey(-stair_offset)
    trench = c << stair_trench
    poly = stair_trench.get_polygons()[gf.get_layer((2, 0))][0]
    sized_poly = poly.sized(dx=10e3, dy=0,mode=1)
    c.add_polygon(sized_poly, layer='DEEP_ETCH')
    c2 = gf.Component()
    beam_mask = gf.boolean(c,c,'not','DEEP_ETCH','DEEP_ETCH','WG')
    y = -3
    for length in length_list:
        for width in width_list_repeat:
            label = text_outline(f'{width}x{length}',size=12, layer=(2,0), outline_width=0.5, with_mask=False)
            label_ref = c2 << label
            label_ref.movex(25)
            label_ref.movey(y)
            label_ref2 = c2 << label
            label_ref2.movex(25+1500)
            label_ref2.movey(y)
            
            label_ref3 = c2 << label
            label_ref3.movex(-length-100)
            label_ref3.movey(y)

            label_ref4 = c2 << label
            label_ref4.movex(25+1500 - length - 150)
            label_ref4.movey(y)
            y += beam_spacing
        y += stair_offset
    c2.add_ref(beam_mask)
    beam_ref2 = c2 << beam_mask
    beam_ref2.movex(1500)
    c2.add_ref(stair_trench)
    return c2
@gf.cell
def resonator_array(gap_list, repeat_num, resonator=1):
    gap_list_repeat = list(repeat(gap_list, repeat_num))
    c = gf.Component()
    x = 0
    y = 0
    for gap_list_ in gap_list_repeat:
        for gap in gap_list_:
            if resonator == 1:
                resonator_ref = c << resonator_ald_test(gap)
            else:
                resonator_ref = c << resonator_ald_test2(gap)
            resonator_ref.move((x,y))
            y += 30
        y=0
        x += 120
    return c
@gf.cell
def comb_drive_array(gap_list, repeat_num):
    gap_list_repeat = list(repeat(gap_list, repeat_num))
    c = gf.Component()
    x = 0
    y = 0
    for gap_list_ in gap_list_repeat:
        for gap in gap_list_:
            comb_drive_ref = c << comb_drive_ald_test(gap)
            comb_drive_ref.move((x,y))
            y += 30
        y=0
        x += 120
    return c
@gf.cell
def beam_mask_single():
    with when(beam_array,"y = -3").do("c2<<beam_mask").goto('return c2'):
        c = beam_array()
    y = -3
    for length in length_list:
        for width in width_list_repeat:
            label = text_outline(f'{width}x{length}',size=12, layer='DEEP_ETCH', outline_width=0.5, with_mask=False)
            label_ref3 = c << label
            label_ref3.movex(-length-100)
            label_ref3.movey(y)

            label_ref2 =c  << label
            label_ref2.movex(25)
            label_ref2.movey(y)
            y += beam_spacing
        y += 10
    return c

2025-10-08 11:26:43.520 | INFO     | kfactory.kcell:show:3948 - klive v0.4.1: Opened file 'c:\Users\xwei2\Documents\Python Projects\NOEMS_Layout\build\gds\2540200948.oas'


In [3]:
c = gf.Component()
# beam_array_ref = c << beam_array()
# beam_array_ref.movey(-1000)

from dowhen import goto
goto('return c').when(big_mark_set, "(c << text_outline")

marker = big_mark_set().copy()
marker.flatten()
marker.remap_layers({(7,0):'WG'},recursive=True)
marker_l = c << marker
marker_r = c << marker
marker_l.movex(-2500)
marker_r.movex(2500)
resonator_gap_list = [0.05, 0.1, 0.15, 0.2, 0.3, 0.4 , 0.5]
resonator_array_ref = c << resonator_array(resonator_gap_list, 5,2)
resonator_array_ref.movey(-500)
resonator_array_ref2 = c << resonator_array(resonator_gap_list, 5,1)
# resonator_array_ref2.movey(-500)
comb_drive_array_ref = c << comb_drive_array(resonator_gap_list, 5)
comb_drive_array_ref.movey(400)
c.add_ref(frame(c.bbox(), layers=((3,6),(1,0))))
c.show()

c:\Users\xwei2\Documents\Python Projects\NOEMS_Layout\.conda\Lib\site-packages\gdsfactory\components\couplers\coupler90.py:42: UserWarning: {'radius': 5} ignored for cross_section 'xs_30f051d8'
  x = gf.get_cross_section(cross_section, radius=radius)
c:\Users\xwei2\Documents\Python Projects\NOEMS_Layout\.conda\Lib\site-packages\gdsfactory\components\couplers\coupler90.py:42: UserWarning: {'radius': 5} ignored for cross_section 'xs_6f54c067'
  x = gf.get_cross_section(cross_section, radius=radius)
c:\Users\xwei2\Documents\Python Projects\NOEMS_Layout\.conda\Lib\site-packages\kfactory\decorators.py:383: UserWarning: Port type structural not in ['optical', 'electrical', 'placement', 'vertical_te', 'vertical_tm', 'vertical_dual', 'electrical_rf', 'pad', 'pad_rf', 'bump', 'edge_coupler']. Please add it to the port_types list in the config gf.CONF.port_types.
  cell = f(**params)  # type: ignore[call-arg]


In [4]:
-c.bbox().center()

0,0

In [17]:
a.enlarge(100).center()

0,0

In [6]:
c2 = gf.Component()
# beam_mask1 = c2 << beam_mask_single()
# beam_mask1.move((-300,-2000))
# beam_mask2 = c2 << beam_mask_single()
# beam_mask2.move((1300,-2000))
beam_mask3 = c2 << beam_mask_single()
beam_mask3.move((-300,-1500))
beam_mask4 = c2 << beam_mask_single()
beam_mask4.move((1300,-1500))
# beam_mask4.rotate(180)
marker = big_mark_set().copy()
marker.flatten()
marker.remap_layers({(7,0):'WG'},recursive=True)
marker_l = c2 << marker
marker_r = c2 << marker
marker_l.movex(-2500)
marker_r.movex(2500)
c2.show()

In [4]:
c3 = gf.Component()

def beam_with_anchor(width, length, support_length, anchor_size:tuple):
    c = gf.Component()
    beam = c << doubly_clamped_beam_with_round_support(width, length, support_length,create_mask=True,mask_offset=3)
    anchor_1 = c << gf.components.rectangle(size=anchor_size, port_type='placement')
    anchor_2 = c << gf.components.rectangle(size=anchor_size, port_type='placement')
    anchor_1.connect("e3", beam.ports["w1"], allow_width_mismatch=True)
    anchor_2.connect("e1", beam.ports["e1"], allow_width_mismatch=True)
    c.flatten()
    return c
@gf.cell
def beam_array():
    c3 = gf.Component()
    stair_offset = 10
    y=0
    for length in length_list:
            for width in width_list_repeat:
                ref = c3.add_ref(beam_with_anchor(width, length, 1, (20, 35)))
                ref.movey(y)
                y += beam_spacing
            y += stair_offset
    y = -3
    for length in length_list:
        for width in width_list_repeat:
            label = text_outline(f'{width}x{length}',size=12, layer='DEEP_ETCH', outline_width=0.5, with_mask=False)
            label_ref3 = c3 << label
            label_ref3.movex(-length-100)
            label_ref3.movey(y)

            label_ref2 =c3  << label
            label_ref2.movex(25)
            label_ref2.movey(y)
            y += beam_spacing
        y += 10
    return c3
c3 = gf.Component()
for i in range(3):
    beam_mask = c3 << beam_array()
    beam_mask.movex(i*1600)
c3.show()

In [6]:
c.write_gds(fr'.\gds_files\{file_name_resonator}.gds',with_metadata=False)

WindowsPath('gds_files/ID61_ALD Resonator test.gds')

In [ ]:
c2.write_gds(fr'.\gds_files\{file_name_beam}.gds',with_metadata=False)


NameError: name 'c2' is not defined